# 1 dimension 1 layer network without recurrent connectivity

In [1]:
# set fileName

fileName = "eg000_small-network"

In [2]:
# set libs

import pyNN.spiNNaker as sim
import numpy as np
import matplotlib.pyplot as plt

Detected PyNN version 0.9.4 and Neo version 0.6.1


In [3]:
# simulation setting

dt         = 1          # (ms)
syn_delay  = 1.0           # (ms)
simtime    = 1000.0        # (ms)
sim.setup(timestep=dt, max_delay=syn_delay)

2021-04-10 14:51:45 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/spinnaker/.spynnaker.cfg
2021-04-10 14:51:45 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries
2021-04-10 14:51:45 INFO: Setting time scale factor to 1.
2021-04-10 14:51:45 INFO: Setting machine time step to 1000 micro-seconds.


['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']


0

In [4]:
# network parameters

n_pop = 20     # Number of cells
weight = 0.05  # synaptic weight (µS)

cell_params = {
    'tau_m'      : 20.0,   # (ms)
    'tau_syn_E'  : 2.0,    # (ms)
    'tau_syn_I'  : 4.0,    # (ms)
    'e_rev_E'    : 0.0,    # (mV)
    'e_rev_I'    : -70.0,  # (mV)
    'tau_refrac' : 2.0,    # (ms)
    'v_rest'     : -60.0,  # (mV)
    'v_reset'    : -70.0,  # (mV)
    'v_thresh'   : -50.0,  # (mV)
    'cm'         : 0.5}    # (nF)


In [5]:
# making the network
cells = sim.Population(n_pop, # number of cells
                       sim.IF_cond_exp, # cell model
                       cell_params, # cell parameters
                       label="cells")


In [6]:
# designing the thalamic input

spike_times = [[]]*n_pop #list of spike lists, where one spike list is related to one spike source
random_sources_idx = [np.random.randint(0, n_pop) for i in range(5)]

for idx, sources in enumerate(random_sources_idx):
    #spike_times[sources] = [np.random.normal(loc=10, scale=0.1) for n in range(10)]
    spike_times[sources] = np.arange(200, 800, 50).tolist()

spike_source = sim.Population(n_pop, sim.SpikeSourceArray(spike_times=spike_times))
spike_source.record('spikes')

input_conns = sim.Projection(spike_source, # from 
                             cells, # to
                             connector = sim.FixedProbabilityConnector(0.25), 
                             synapse_type = sim.StaticSynapse(weight=weight, delay=syn_delay),
                             receptor_type = 'excitatory')

fig, axes = plt.subplots(1,1)
axes_list = fig.axes
axes_list[0].eventplot(spike_times)
axes_list[0].set_xlabel('[ms]')
axes_list[0].set_ylabel('thalamic spike sources')
axes_list[0].set_title('rasterplot')
axes_list[0].set_xlim(0, simtime)
print('selected random sources', random_sources_idx)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

selected random sources [11, 9, 0, 0, 6]


In [7]:
# record state variables

cells.record(['spikes'])
cells.record(['v', 'gsyn_exc', 'gsyn_inh'])

In [8]:
# run simulation

sim.run(simtime)

2021-04-10 14:51:50 INFO: Simulating for 1000 1.0ms timesteps using a hardware timestep of 1000us
2021-04-10 14:51:50 INFO: Starting execution process
2021-04-10 14:51:53 INFO: Time 0:00:03.462629 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2021-04-10 14:52:03 INFO: Time 0:00:10.178018 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2021-04-10 14:52:09 INFO: Time 0:00:05.393291 taken by PartitionAndPlacePartitioner
Created spalloc job 5927704
2021-04-10 14:52:09 INFO: Created spalloc job 5927704
Waiting for board power commands to complete.
2021-04-10 14:52:09 INFO: Waiting for board power commands to complete.
2021-04-10 14:52:14 INFO: Time 0:00:05.049379 taken by SpallocAllocator
2021-04-10 

1000.0

In [9]:
# save results

outputs = cells.get_data() # on notebook space
cells.write_data(fileName + '.pkl', "spikes") # on folder

Getting spikes for cells
|0%                          50%                         100%|
Getting v for cells
|0%                          50%                         100%|
Getting gsyn_exc for cells
|0%                          50%                         100%|
Getting gsyn_inh for cells
|0%                          50%                         100%|
Getting spikes for cells
|0%                          50%                         100%|


In [10]:
# recovering results

def recover_results(outputs):
    results = {}
    
    for analogsignal in outputs.segments[0].analogsignals:
        print(analogsignal.name)
        # to get voltage and conductances
        results[analogsignal.name] = analogsignal
        # to get spikes
        results['spikes'] = outputs.segments[0].spiketrains
    return results

In [11]:
# check results

results = recover_results(outputs)
results.keys()

v
gsyn_exc
gsyn_inh


dict_keys(['v', 'spikes', 'gsyn_exc', 'gsyn_inh'])

In [12]:
# check the mean firing rate

print("Mean firing rate: ", cells.mean_spike_count() * 1000.0 / simtime, "Hz")

Getting spikes for cells
|0%                          50%                         100%|

Mean firing rate:  3.6 Hz


In [13]:
# close simulation

sim.end()

In [14]:
# check the spikes

fig, axes = plt.subplots(1, 1)
fig.tight_layout(pad=5)
axes_list = fig.axes

axes_list[0].eventplot(results['spikes'])
axes_list[0].set_title('rasterplot of cells')
axes_list[0].set_xlabel('[ms]')
axes_list[0].set_ylabel('cells')
axes_list[0].set_xlim(0, simtime)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 1000.0)

In [15]:
random_sources_idx

[11, 9, 0, 0, 6]

In [16]:
# check the voltage

pickedCell = random_sources_idx[2]

fig, axes = plt.subplots(1, 1)
fig.tight_layout(pad=3)

axes_list = fig.axes
axes_list[0].plot(results['v'].T[pickedCell], label='cell_' + str(pickedCell))

axes_list[0].set_title('voltage of cells')
axes_list[0].set_xlabel('time [ms]')
axes_list[0].set_ylabel('[mV]')
axes_list[0].legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
# check the conductance

pickedCell = random_sources_idx[2]



fig, axes = plt.subplots(2, 1)
fig.tight_layout(pad=5)

axes_list = fig.axes

for idx, conductance in enumerate(['gsyn_exc', 'gsyn_inh']):

    fig.tight_layout(pad=3)
    axes_list = fig.axes
    axes_list[idx].plot(results[conductance].T[pickedCell], label='cell_' + str(pickedCell))
    axes_list[idx].set_title(conductance)
    axes_list[idx].set_xlabel('time [ms]')
    axes_list[idx].set_ylabel('[uS]')
    axes_list[idx].legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## task 1: on state variables behaviour

- why the gsyn_inh (conductance related to cell inhibition) is equal to zero?

## task 2: on neuronal parameters

- change the tau_m (membrane time constant) parameter and have a look at the changes on the voltage trace.

- change other neuronal parameters and investigate their effect on the network behaviour

- info about parameters http://neuralensemble.org/docs/PyNN/standardmodels.html